This notebook provides examples to go along with the [textbook](https://underactuated.csail.mit.edu/humanoids.html).  I recommend having both windows open, side-by-side!


In [ ]:
import numpy as np
from math import cos, sin, pi, atan2, sqrt
import pydot
from functools import partial
import csv
import time

from pydrake.all import (
    AddDefaultVisualization,
    DiscreteContactApproximation,
    PidController,
    RobotDiagramBuilder,
    Simulator,
    StartMeshcat,
    namedview,
    MathematicalProgram,
    AddUnitQuaternionConstraintOnPlant,
    OrientationConstraint,
    PositionConstraint,
    RigidTransform,
    RotationMatrix,
    eq,
    SnoptSolver,
    Solve,
    AutoDiffXd,
    ExtractGradient,
    ExtractValue,
    JacobianWrtVariable,
    InitializeAutoDiff,
    PiecewisePolynomial,
    MeshcatVisualizer
)
from IPython.display import SVG

from underactuated import ConfigureParser, running_as_notebook
from underactuated.multibody import MakePidStateProjectionMatrix

from spot_ik_helpers import SpotStickFigure

from gait_optimization import gait_optimization

In [ ]:
# Start the visualizer (run this cell only once, each instance consumes a port)
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at https://a45cd6cf-c8cd-4212-9b4f-261528e6e6af.deepnoteproject.com/7000/
Installing NginX server for MeshCat on Deepnote...


# Spot

[Spot](https://bostondynamics.com/products/spot/) is a robot built by BostonDynamics.
The following Spot urdf was modified such that it contains frames for the four feet.

In [ ]:
def load_urdf_file(file_path):
    with open(file_path, 'r') as file:
        urdf_string = file.read()
    return urdf_string

# Example usage:
spot_urdf_path = 'spot-20240430-141158.urdf'
# urdf_path = 'spot.urdf'
spot_urdf_string = load_urdf_file(spot_urdf_path)
#print(urdf_string)

ground_path = 'ground.urdf'
ground_string = load_urdf_file(ground_path)


## Generate Terrains

Terrains are loaded from txt file (output of GCS file) and box URDFS are generated according to x,y positions and length, width, height of the boxes. 

In [ ]:
import random

def create_random_box():
    # Generate random dimensions for the box
    length = random.uniform(0.1, 1.0)
    width = random.uniform(0.1, 1.0)
    height = random.uniform(0.1, 1.0)
    return length, width, height

def create_random_location():
    # Generate random x and y coordinates for the box
    x = random.uniform(-2.0, 2.0)
    y = random.uniform(-2.0, 2.0)
    return x, y

def generate_urdf_string(name, length, width, height, x, y):
    urdf = f"""<?xml version='1.0' encoding='us-ascii'?>
<robot name={name}>
    <link name={name}>
        <visual>
            <origin xyz="{x} {y} {height/2}" rpy="0 0 0" />
            <geometry>
                <box size="{length} {width} {height}" />
            </geometry>
            <material name="white">
                <color rgba="1 1 1 1" />
            </material>
        </visual>
        <collision>
            <origin xyz="{x} {y} {height/2}" rpy="0 0 0" />
            <geometry>
                <box size="{length} {width} {height}" />
            </geometry> 
            <surface>
                <friction>
                <ode>
                    <mu>0.5</mu>  <!-- Coefficient of friction -->
                    <mu2>0.5</mu2> <!-- Optional anisotropic friction coefficient -->
                </ode>
                </friction>
            </surface>
        </collision>
        <inertial>
            <mass value="1.8330189874700853" />
            <inertia ixx="0.1" ixy="0" ixz="0" iyy="0.1" iyz="0" izz="0.1" />
            <origin xyz="{x} {y} {height/2}" rpy="0 0 0" />
        </inertial>
    </link>
</robot>
"""
    return urdf

length, width, height = create_random_box()
x, y = create_random_location()
box_urdf = generate_urdf_string('"test"',length, width, height, x, y)
print("Random box URDF:")
print(box_urdf)


Random box URDF:
<?xml version='1.0' encoding='us-ascii'?>
<robot name="test">
    <link name="test">
        <visual>
            <origin xyz="-1.3566704453609195 0.3093785024695803 0.49756801881012785" rpy="0 0 0" />
            <geometry>
                <box size="0.1755486518812613 0.22676454424945375 0.9951360376202557" />
            </geometry>
            <material name="white">
                <color rgba="1 1 1 1" />
            </material>
        </visual>
        <collision>
            <origin xyz="-1.3566704453609195 0.3093785024695803 0.49756801881012785" rpy="0 0 0" />
            <geometry>
                <box size="0.1755486518812613 0.22676454424945375 0.9951360376202557" />
            </geometry> 
            <surface>
                <friction>
                <ode>
                    <mu>0.5</mu>  <!-- Coefficient of friction -->
                    <mu2>0.5</mu2> <!-- Optional anisotropic friction coefficient -->
                </ode>
                </fric

In [ ]:
import pickle
def add_boxes_to_terrain_from_file(filename, parser, plant):
    f = open(filename, "rb")
    terrain_dict = pickle.load(f)
    f.close()

    for stone_name in terrain_dict:
        x = terrain_dict[stone_name]["x"]
        y = terrain_dict[stone_name]["y"]
        length = terrain_dict[stone_name]["length"]
        width = terrain_dict[stone_name]["width"]
        height = terrain_dict[stone_name]["height"]

        box_urdf = generate_urdf_string(f'"{stone_name}"', length, width, height, x, y)
        parser.AddModelsFromString(box_urdf, "urdf")

        plant.WeldFrames(
            plant.world_frame(), # Attach to world frame
            plant.GetFrameByName(f'{stone_name}'), # Attach to box grame
            RigidTransform([0,0,0]) # Weld location
        )

## Get footstep plan from GCS

In [ ]:
pos_left = np.array([[0.5,        0.8        ],
                    [0.85,       0.89146762],
                    [1.24167495, 0.88140645],
                    [1.24167495, 0.88140645],
                    [1.61797827, 0.87281328],
                    [1.87474689, 0.68544733],
                    [2.,         0.5       ]])

pos_right = np.array([[ 5.00000000e-01,  5.00000000e-01],
                    [ 8.93165085e-01,  5.74097808e-01],
                    [ 1.08891384e+00,  5.72398666e-01],
                    [ 1.41705050e+00,  5.67837359e-01],
                    [ 1.74694647e+00,  3.77585899e-01],
                    [ 2.07449412e+00,  1.93001620e-01],
                    [ 2.00000000e+00, -1.13021754e-15]])

# GENEREATE FOOTSTEPS
# Order: RB, RF, LF, LB
init_step = np.array([
    pos_right[0], # RB
    pos_right[2], # RF
    pos_left[2], # LF
    pos_left[0] # LB
])
footsteps = [init_step]
foot_indices = [] # LF, RF, LB, RB
foot_IK_indices = [2, 1, 3, 0] # conversion of opt indices to IK indices

for i in range(pos_left.shape[0]-3):
    step_lf = np.array([
        pos_right[i], # RB
        pos_right[i+2], # RF
        pos_left[i+3], # LF
        pos_left[i] # LB
    ])
    step_rb = np.array([
        pos_right[i+1], # RB
        pos_right[i+2], # RF
        pos_left[i+3], # LF
        pos_left[i] # LB
    ])
    step_rf = np.array([
        pos_right[i+1], # RB
        pos_right[i+3], # RF
        pos_left[i+3], # LF
        pos_left[i] # LB
    ])
    step_lb = np.array([
        pos_right[i+1], # RB
        pos_right[i+3], # RF
        pos_left[i+3], # LF
        pos_left[i+1] # LB
    ])
    
    footsteps.append(step_lf)
    footsteps.append(step_rb)
    footsteps.append(step_rf)
    footsteps.append(step_lb)

    foot_indices.append(0)
    foot_indices.append(3)
    foot_indices.append(1)
    foot_indices.append(2)


## Execute footstep plan trajectory

In [ ]:
def execute_traj():
    #env setup
    robot_builder = RobotDiagramBuilder(time_step=1e-4)

    parser = robot_builder.parser()
    ConfigureParser(parser)
    parser.AddModelsFromString(ground_string,".urdf" )
    parser.AddModelsFromString(spot_urdf_string,".urdf" )
    plant = robot_builder.plant()
    plant.set_discrete_contact_approximation(DiscreteContactApproximation.kLagged)
    add_boxes_to_terrain_from_file("/work/terrains/promising_terrain.txt", parser, plant)
    box_height = 0.1


    plant.Finalize()
    builder = robot_builder.builder()

    num_u = plant.num_actuators()
    kp = 1000 * np.ones(num_u)
    ki = 0 * np.ones(num_u)
    kd = 100 * np.ones(num_u)

    S = MakePidStateProjectionMatrix(plant)
    control = builder.AddSystem(
        PidController(
            kp=kp,
            ki=ki,
            kd=kd,
            state_projection=S,
            output_projection=plant.MakeActuationMatrix()[6:, :].T,
        )
    )

    builder.Connect(
        plant.get_state_output_port(), control.get_input_port_estimated_state()
    )
    builder.Connect(control.get_output_port(), plant.get_actuation_input_port())

    # visualizer = MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)
    AddDefaultVisualization(builder, meshcat=meshcat)

    spot = plant.GetModelInstanceByName("spot")
    diagram = robot_builder.Build()
    simulator = Simulator(diagram)
    context = simulator.get_mutable_context()
    plant_context = plant.GetMyContextFromRoot(context)
    control_context = control.GetMyContextFromRoot(context)

    # SET INITIAL POSITION
    foot_init = footsteps[0]
    # IK takes 3D pos with negated z
    foot_init_IK_frame = np.hstack((foot_init[:,0].reshape(4,1), np.zeros((4,1)), -1*foot_init[:,1].reshape(4,1)))
    mean_x = np.mean(foot_init[:,0])
    mean_z = np.mean(foot_init[:,1])

    # compute body orientation (psi)
    right_vec = foot_init[1,:] - foot_init[0,:] # RF - RB
    left_vec = foot_init[2,:] - foot_init[3,:] # LF - LB
    right_psi = atan2(right_vec[1], right_vec[0])
    left_psi = atan2(left_vec[1], left_vec[0])
    mean_psi = (right_psi + left_psi)/2 # average orientation of R/L feet vectors

    sm = SpotStickFigure(x=mean_x, z=-mean_z, psi=mean_psi) # negate z for IK
    sm.set_absolute_foot_coordinates(foot_init_IK_frame)
    rb, rf, lf, lb = sm.get_leg_angles()

    q0 = plant.GetPositions(plant_context)
    q0[4] = mean_x
    q0[5] = mean_z
    q0[6] = sm.y + box_height + 0.05 # add height so spot dropped
    q0[7:10] = np.array(lf)
    q0[10:13] = -np.array(rf)
    q0[13:16] = np.array(lb)
    q0[16:19] = -np.array(rb)
    plant.SetPositions(plant_context, q0)

    #if want to enforce inital position
    #x0 = S @ plant.get_state_output_port().Eval(plant_context)
    #control.get_input_port_desired_state().FixValue(control_context, x0)

    # SIMULATION
    meshcat.StartRecording()
    total_t = 0
    solve_times = []

    for i, next_foot in enumerate(footsteps[1:10]):
        foot_idx = foot_indices[i]

        # Solve optimization
        print(f"Step {i+1}")
        start_time = time.time()
        t_sol, q_sol, v_sol,q_end = gait_optimization(plant, plant_context, spot, next_foot, foot_idx, box_height)
        elapsed = time.time() - start_time
        print(f"Solve time: {elapsed}")
        solve_times.append(elapsed)

        # Animate trajectory
        simulator.set_target_realtime_rate(1.0 if running_as_notebook else 0)
        context = simulator.get_mutable_context()
        plant_context = plant.GetMyContextFromRoot(context)
        control_context = control.GetMyContextFromRoot(context)

        PositionView = namedview(
            "Positions", plant.GetPositionNames(spot, always_add_suffix=False)
        )
        VelocityView = namedview(
            "Velocities", plant.GetVelocityNames(spot, always_add_suffix=False)
        )

        t0 = t_sol[0] + total_t
        tf = t_sol[-1] +  total_t
        delta_t = 1/32
        for t in (np.arange(t0, tf, delta_t)):
            ts = (t - t0) % (tf - t0)
            q_t = PositionView(q_sol.value(ts))
            v_t = VelocityView(v_sol.value(ts))
            x_des = np.vstack((q_t[7:],v_t[6:]))

            # TELEPORT
            # context.SetTime(t)
            # plant.SetPositions(plant_context, q_t[:])
            # plant.SetVelocities(plant_context, v_t[:])
            # diagram.ForcedPublish(context)

            # PD
            control.get_input_port_desired_state().FixValue(control_context, x_des)
            simulator.AdvanceTo(t)

        t_add = 1.5
        v_t=np.zeros((12,))
        x_final=np.hstack((q_end[7:], v_t))
        for t_next in np.arange(tf, tf+t_add, delta_t):
            control.get_input_port_desired_state().FixValue(control_context, x_final)
            simulator.AdvanceTo(t_next)

        total_t = tf + t_add

    simulator.AdvanceTo(total_t)

    # visualizer.PublishRecording()
    meshcat.PublishRecording()

    return solve_times

solve_times = execute_traj()

Step 1
SNOPT
True
Solve time: 11.135178327560425
Step 2
SNOPT
True
Solve time: 20.543782949447632
Step 3
SNOPT
True
Solve time: 8.037681579589844
Step 4
SNOPT
True
Solve time: 10.256193399429321
Step 5
SNOPT
True
Solve time: 8.460823774337769
Step 6
SNOPT
True
Solve time: 9.764229536056519
Step 7
SNOPT
True
Solve time: 24.11404323577881
Step 8
SNOPT
True
Solve time: 16.954559564590454
Step 9
SNOPT
True
Solve time: 8.761406183242798


In [ ]:
times = np.array(solve_times)
print(f"Mean solve time: {np.mean(times)}")
print(f"Stddev solve time: {np.std(times)}")

Mean solve time: 15.703105688095093
Stddev solve time: 7.681897723551365


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=a45cd6cf-c8cd-4212-9b4f-261528e6e6af' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>